# Hospital 

In [16]:
import pandas as pd

In [17]:
hospital_location = pd.read_csv('../data/raw/hospital_name.csv')
property_data_2017_2022 = pd.read_csv('../data/raw/2017_2022_house_data.csv')
property_data_2023_2025 = pd.read_csv('../data/raw/23To25_property.csv')
po_sa2 = pd.read_csv('../data/raw/postcode_sa2.csv')

In [18]:
property_data_2023_2025 = property_data_2023_2025[['Unnamed: 0', 'name', 'property_type', 'year', 'coordinates',
       'postcode', 'sa2_coor', 'closest_sa2', 'bed', 'bath', 'car', 'Income',
       'Population', 'real_school_distance', 'real_school_duration',
       'real_station_distance', 'real_station_duration', 'week_rent']]


In [19]:
postcode = sorted(set(hospital_location['Postcode']))

In [20]:
hospital_location = hospital_location.groupby('Postcode').count()[['Formal Name']]

In [21]:
hospital_location['Postcodes'] = postcode


In [22]:
hospital_location.rename({"Formal Name":'Hospital num'},axis=1, inplace=True)

In [23]:
def add_sa2(row,sa2_dataframe):
    postcode = row['Postcodes']
    selected_postcode = sa2_dataframe.loc[sa2_dataframe['postcodes'] == postcode]
    sa2 = [(selected_postcode.iloc[i])["SA2_names"] for i in range(len(selected_postcode))]
    sa2=list(set(sa2))
    return sa2

In [24]:
hospital_location["sa2_region"] = hospital_location.apply(lambda row: add_sa2(row,po_sa2), axis=1)

In [25]:
# Merge the number of hospital near by
property_data_2017_2022 = pd.merge(property_data_2017_2022,hospital_location, right_on='Postcodes',left_on = 'postcode',how='left')
property_data_2023_2025 = pd.merge(property_data_2023_2025,hospital_location, right_on='Postcodes',left_on = 'postcode',how='left')

In [26]:
property_data_2017_2022 = property_data_2017_2022.drop(['Postcodes'],axis=1)
property_data_2017_2022 = property_data_2017_2022.drop(['sa2_region'],axis=1)
property_data_2017_2022 = property_data_2017_2022.drop(['Unnamed: 0.1'],axis=1)
property_data_2017_2022 = property_data_2017_2022.drop(['Unnamed: 0'],axis=1)

In [27]:
property_data_2023_2025 = property_data_2023_2025.drop(['Postcodes'],axis=1)
property_data_2023_2025 = property_data_2023_2025.drop(['sa2_region'],axis=1)
property_data_2023_2025 = property_data_2023_2025.drop(['Unnamed: 0'],axis=1)

In [28]:
property_data_2017_2022['Hospital num'] =  property_data_2017_2022['Hospital num'].fillna(0)
property_data_2023_2025['Hospital num'] =  property_data_2023_2025['Hospital num'].fillna(0)

In [29]:
property_data_2017_2022.to_csv('../data/raw/2017_2022_house_data_final.csv')
property_data_2023_2025.to_csv('../data/raw/2023_2025_house_data_final.csv')